In [1]:
# .env에 있는 환경변수 불러오기
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
# 모델 만들기
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
  model = "gpt-4o-mini",
  temperature = 0.5
)

model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002729E6AD700>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002729E30F0B0>, root_client=<openai.OpenAI object at 0x000002729CF17CE0>, root_async_client=<openai.AsyncOpenAI object at 0x000002729E2A6DE0>, model_name='gpt-4o-mini', temperature=0.5, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [ ]:
from langchain_core.prompts import PromptTemplate
# chain = prompt | model | output_parser
# chain 만드는 함수 만들기
def make_chain(template, output_parser):
  """chain을 만드는 함수"""
  prompt = PromptTemplate.from_template(template)
  chain = prompt | model | output_parser

  return chain


## 회의록 음성 텍스트 불러오기


In [ ]:
# 파일을 불러오는 방법
with open('output/meeting_transcript.txt', "r") as f:
  meeting_transcript = f.read()

print(meeting_transcript)

 다들 도착하셨나요? 그럼 회의 시작할게요.
 먼저 디자인 팀 상황부터 공유해주시겠어요?
 네, 디자인 팀은 이번 주까지 시안 1차 수정본 제출목별로 작업 중입니다.
 주요 피드백 반영했고, 마감은 목을까지 가능합니다.
 좋아요. 개발팀은요?
 기능 개발은 80% 열됐습니다.
 요근 기능과 계시판 기능은 이번 주에 마무리할 예정이고
 다음 주 월을 붙어 내부 테스트를 시작하려고 합니다.
 일정대로 잘 진행되고 있네요.
 마케팅 쪽은 어떤가요?
 마케팅 팀은 월 초에 있을 프로모션을 준비 중입니다.
 다만 이번 주 안으로 기획 초원을 작성하는 건 조금 타이트할 것 같은데
 가능하다면 기획 초원 제출 기안을 다음 주 수요일까지로 도정해 주실 수 있을까요?
 음, 알겠어요. 전체 일정에는 영향이 크지 않으니
 다음 주 수요일까지로 조정합시다.
 그럼 지금 남은 과제는 기능 테스트 계획 수리
 디자인 수정 최종본 검토 마케팅 기획 초원 전검 2세 가지로 정리할 수 있겠네요?
 네, 테스트 계획 문서는 이번 주 금요일까지 작성해서 공유하겠습니다.
 디자인 수정보는 목요일에 완성되는 대로 바로 공유 드리겠습니다.
 좋아요. 마지막으로 다음 회의 일정을 잡을게요.
 3월 29일 월을 오전 10시에 모두 괜찮으세요?
 죄송한데 그날 오전에 내부 회의가 있어서 11시 2으로 밀어주실 수 있을까요?
 저는 괜찮습니다.
 저도 11시이면 문제 없습니다.
 알겠습니다.
 그럼 다음 회의는 4월 29일 월을 오전 11시로 확정하겠습니다.
 오늘 회의는 여기까지 하겠습니다. 모두 수고하셨습니다.



## 1. 회의록 요약

In [14]:
template = """
너는 회의록 요약 전문가야.
다음 회의록을 읽고, 아래와 같은 형식으로 정리해줘.
[회의록 요약 양식]
- 회의 일시:  
- 회의 주제:  
- 내용 요약: 목록형 리스트로 표현
- 팀 별 요약:
  - [팀 1]: 목록형 리스트로 표현
  - [팀 2]: 목록형 리스트로 표현
  - ...
...
[회의록]
{meeting_transcript}
"""

In [15]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = make_chain(template, output_parser)

chain

PromptTemplate(input_variables=['meeting_transcript'], input_types={}, partial_variables={}, template='\n너는 회의록 요약 전문가야.\n다음 회의록을 읽고, 아래와 같은 형식으로 정리해줘.\n[회의록 요약 양식]\n- 회의 일시:  \n- 회의 주제:  \n- 내용 요약: 목록형 리스트로 표현\n- 팀 별 요약:\n  - [팀 1]: 목록형 리스트로 표현\n  - [팀 2]: 목록형 리스트로 표현\n  - ...\n...\n[회의록]\n{meeting_transcript}\n')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002729E6AD700>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002729E30F0B0>, root_client=<openai.OpenAI object at 0x000002729CF17CE0>, root_async_client=<openai.AsyncOpenAI object at 0x000002729E2A6DE0>, model_name='gpt-4o-mini', temperature=0.5, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [16]:
response = chain.invoke(
  {
    "meeting_transcript": meeting_transcript
  }
)

print(response)

[회의록 요약 양식]  
- 회의 일시: 2023년 3월 27일  
- 회의 주제: 프로젝트 진행 상황 및 일정 조정  
- 내용 요약:  
  - 디자인 팀은 시안 1차 수정본 제출을 위해 작업 중이며, 주요 피드백을 반영하고 목요일까지 마감 가능.  
  - 개발팀은 기능 개발이 80% 완료되었으며, 요근 기능과 계시판 기능을 이번 주에 마무리하고 다음 주 월요일에 내부 테스트 시작 예정.  
  - 마케팅 팀은 월 초 프로모션 준비 중이며, 기획 초안 제출 기한을 다음 주 수요일로 조정 요청. 전체 일정에는 큰 영향 없음.  
  - 남은 과제: 기능 테스트 계획 수립, 디자인 수정 최종본 검토, 마케팅 기획 초안 전검.  
  - 테스트 계획 문서는 이번 주 금요일까지 작성 및 공유, 디자인 수정본은 목요일에 공유 예정.  
  - 다음 회의 일정은 4월 29일 월요일 오전 11시로 확정.  

- 팀 별 요약:  
  - [디자인 팀]:  
    - 시안 1차 수정본 제출 작업 중  
    - 주요 피드백 반영 완료  
    - 목요일까지 마감 가능  
  - [개발 팀]:  
    - 기능 개발 80% 완료  
    - 요근 기능과 계시판 기능 이번 주 마무리 예정  
    - 다음 주 월요일 내부 테스트 시작 예정  
  - [마케팅 팀]:  
    - 월 초 프로모션 준비 중  
    - 기획 초안 제출 기한을 다음 주 수요일로 조정 요청  
    - 전체 일정에 큰 영향 없음  


## 2. To-do List 생성


In [17]:
template = """
너는 회의록에서 해야 할 일을 정리하는 비서야.
회의정보에는 내 이름과 부서가 포함되어 있어. 
이를 참고하여 아래 회의록을 읽고, 부서에 맞는 내가 할 일을 간단히 작성해줘.
[형식]
항목들을 콤마(,)로 구분하여 작성
[회의 정보]
{meeting_info}
[회의록]
{meeting_transcript}
"""

In [19]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
chain = make_chain(template, output_parser)

response = chain.invoke(
  {
    "meeting_info": {"name": "김나래", "department": "개발팀"},
    "meeting_transcript": meeting_transcript
  }
)

print(response)

['기능 개발 마무리', '내부 테스트 계획 수립', '기능 테스트 계획 문서 작성', '디자인 수정 최종본 검토', '다음 회의 일정 확인']


## 3. 일정 추출

In [20]:
template = """
너는 회의록 중 날짜와 시간 정보를 정리하는 역할이야.
회의 정보를 보고, 오늘 회의를 기준일로 설정하고,
회의록을 읽고, 일정 정보를 아래와 같은 JSON 형식으로 정리해줘.
[형식 예시]
[
  {{
     "title": "회의",
     "start": "2025-05-01T10:00:00",
     "end": "2025-05-05T11:00:00",
     "allDay": True
  }}
]
[회의 정보]
{meeting_info}
[회의록]
{meeting_transcript}
"""

In [21]:
from langchain_core.output_parsers import JsonOutputParser

output_parser = JsonOutputParser()
chain = make_chain(template, output_parser)

response = chain.invoke(
  {
    "meeting_info" : {"name": "김나래", "deapartment": "개발팀", "meeting_date": "2025-04-29 Tuesday"},
    "meeting_transcript": meeting_transcript
  }
)

print(response)

[{'title': '회의', 'start': '2025-04-29T11:00:00', 'end': '2025-04-29T12:00:00', 'allDay': False}]
